# 0. Variables

In [1]:
from correos_automaticos.classes.sharepoint_manager import Sharepoint
from correos_automaticos.classes.file_manager import FileManager
from dotenv import load_dotenv
import os
import re
import pandas as pd
from icecream import ic
import urllib.parse

#script_dir = os.path.dirname(__file__) # for .py files
script_dir = os.getcwd()  # for jupyter

# Cargar variables de entorno
load_dotenv()

# Variables globales
DOWNLOAD_PATH = os.path.join(script_dir, "..", "descargas")  # Carpeta de descargas
UPLOAD_PATH = os.path.join(script_dir, "..", "descargas", "clasificados")  # Carpeta desde donde se subirán archivos
TEMPLATES_PATH = os.path.join(script_dir, "..", "email_templates") # Carpeta desde la que se obtendrán los email templates

# Credenciales segundo usuario
SHAREPOINT_URL_SITE = os.getenv("SHAREPOINT_URL_SITE") # Ruta fija (Enlace)
SHAREPOINT_SITE_NAME = os.getenv("SHAREPOINT_SITE_NAME")
SHAREPOINT_FOLDER = os.getenv("SHAREPOINT_FOLDER") # Ruta del canal (execu compartidos/AOI Tendencias)
SHAREPOINT_DOC = os.getenv("SHAREPOINT_DOC") # Ruta específica del folder (Prueba)
SHAREPOINT_USERNAME = os.getenv("SHAREPOINT_USERNAME") # no usar os.path.join()

# Otras carpetas
#custom_folder_path = f'{SHAREPOINT_USERNAME}/Documents'
excel_name = "Registro de participación DNPE 1.xlsx"

# Iniciar sesión
session = Sharepoint()
session._auth()

Autenticación exitosa. Conexión establecida con SharePoint para https://ceplangobpe.sharepoint.com/sites/DNPE


# 1. Crear excel con los datos de la encuesta

## 1.1. Obtener adjuntos (metadata)

In [2]:
sharepoint_metadata = session.list_files(personal=True)
ic(sharepoint_metadata[:2])

Archivos presentes en la carpeta "Question":
Total de archivos encontrados: 218
 - Propuesta PEI Fase 03_CEPLAN_Milagros Estrada Ram.xlsx
 - Invitación_OFICIO 947-2024-GORE-ICA-SGRH_Erika Celiz Ygnacio.pdf
 - Propuesta PEI Fase 02_Ejemplo CEPLAN_Milagros Estrada Ram.xlsx
 - Correo_ Marco Francisco Torres - Outlook - Dr_Marco Francisco Torr.pdf
 - Reunión sobre convenio de Contraloría y Cepla_Milagros Estrada Ram.pdf
 - ENCC vCEPLAN_REV.DNSE_DNPE_Milagros Estrada Ram.pdf
 - Reunión 6 de noviembre 3 pm_PN ENCC_Milagros Estrada Ram.pdf
 - Reunión de coordinación de la SUNEDU y el CEP_Yiem Ataucusi Ataucu.pdf
 - 1. ENTREGABLE 1-PNP_subsanado CCCRevCBQ DNPE_Carmen del Rosario B.docx
 - Correo recepciónGore Lima PDRC_Carmen del Rosario B.pdf
 - Lista de asistencia PEI 12.08_Milagros Estrada Ram.xlsx
 - LISTA DE ASISTENCIA_TALLER_ONPE_FASE_1_20_AGO_Alvaro Gamboa Buendi.pdf
 - 30.05 Anexo 1  - Programa propuesto Taller li_Yiem Ataucusi Ataucu.docx
 - Coordinación PCD_Erika Celiz Ygnacio.pdf
 -

ic| sharepoint_metadata[:2]: [{'author': SP.User,
                               'editor': None,
                               'name': 'Propuesta PEI Fase 03_CEPLAN_Milagros Estrada Ram.xlsx',
                               'server_relative_url': '/personal/msuarez_ceplan_gob_pe/Documents/Aplicaciones/Microsoft '
                                                      'Forms/Registro de Asistencia Técnica Participación '
                                                      'de la/Question/Propuesta PEI Fase 03_CEPLAN_Milagros '
                                                      'Estrada Ram.xlsx',
                               'time_created': '2024-09-02 17:11:12',
                               'time_last_modified': '2024-09-02 17:12:41',
                               'uniqueId': 'b036086f-a4a1-4bd9-b757-010235905ddd'},
                              {'author': SP.User,
                               'editor': None,
                               'name': 'Invitación_OFICIO 947-202

[{'name': 'Propuesta PEI Fase 03_CEPLAN_Milagros Estrada Ram.xlsx',
  'server_relative_url': '/personal/msuarez_ceplan_gob_pe/Documents/Aplicaciones/Microsoft Forms/Registro de Asistencia Técnica Participación de la/Question/Propuesta PEI Fase 03_CEPLAN_Milagros Estrada Ram.xlsx',
  'time_created': '2024-09-02 17:11:12',
  'time_last_modified': '2024-09-02 17:12:41',
  'author': SP.User,
  'editor': None,
  'uniqueId': 'b036086f-a4a1-4bd9-b757-010235905ddd'},
 {'name': 'Invitación_OFICIO 947-2024-GORE-ICA-SGRH_Erika Celiz Ygnacio.pdf',
  'server_relative_url': '/personal/msuarez_ceplan_gob_pe/Documents/Aplicaciones/Microsoft Forms/Registro de Asistencia Técnica Participación de la/Question/Invitación_OFICIO 947-2024-GORE-ICA-SGRH_Erika Celiz Ygnacio.pdf',
  'time_created': '2024-10-04 15:03:54',
  'time_last_modified': '2024-10-04 15:04:13',
  'author': SP.User,
  'editor': None,
  'uniqueId': 'e4c870ee-5726-44c2-9f78-0166acdbbe55'}]

## 1.2. Obtener df con los resultados de la encuesta

In [4]:
# Para descargar los archivos
#session.list_files(custom_folder_path=custom_folder_path, personal=True)
#session.download_file(excel_name, custom_folder_path=custom_folder_path, personal = True)

# open pandas
excel_path = os.path.join(DOWNLOAD_PATH, "no_clasificados", excel_name)
df = pd.read_excel(excel_path)
df.head()


,Id,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Seleccione la actividad operativa o tema relacionado,Especialista de la DNPE a cargo,Fecha de ejecución de la actividad,Fecha de fin de la actividad (si aplica),Modalidad,...,Número de espacios de acompañamiento y/o revisión durante el mes,Dirección de la persona de contacto/coordinación1,Nombre de la persona de contacto/coordinación1,N° de informe emitido mediante SGD1,Dificultades presentadas1,Oportunidades de mejora1,Información adicional de la asistencia técnica / capacitación y/o difusión y/o exposición1,Adjuntar programa,Adjuntar ppts,Adjuntar otras evidencias
0,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Enrique Del Águila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
1,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,3.0,DNCP,Isabel Rocio Uriona Diaz,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
2,6,2024-06-25 00:02:35,2024-06-25 00:11:11,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-25,NaT,Virtual,...,1.0,DNCP,Roy Gerson Muñoz Gomez,NaN,Disponibilidad de tiempo limitado/insuficiente;,Se recomienda que la revisión del Proyecto de ...,Se dejo comentario al documento que contiene e...,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
3,7,2024-06-25 00:11:17,2024-06-25 00:19:25,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-04-09,NaT,Presencial,...,1.0,DNCP,Litz Tello Flores,NaN,Disponibilidad de tiempo limitado/insuficiente;,NaN,Comentarios generales a la propuesta de cambio...,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
4,8,2024-06-25 00:27:38,2024-06-25 00:29:00,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-04-24,NaT,Presencial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN


# 1.3. Funciones para limpiar el df

In [6]:
df_encuesta = df
sharepoint_metadata_prueba = sharepoint_metadata

def normalize_and_split_links(df, column):
    df = df.copy()
    # Dividimos los enlaces múltiples en filas individuales
    df = df.assign(**{column: df[column].str.split('; ')}).explode(column)
    # Eliminamos espacios al inicio/final de cada enlace
    df[column] = df[column].str.strip()
    return df


def limpiar_url(url):
    # Decodificar la URL
    url_decodificada = urllib.parse.unquote(url)
    
    # Si el enlace contiene '_layouts/15/Doc.aspx', intentamos reconstruirlo
    if "_layouts/15/Doc.aspx" in url_decodificada:
        # Extraemos el archivo (file=) y reconstruimos el enlace
        file_match = re.search(r'file=([^&]+)', url_decodificada)
        if file_match:
            archivo = file_match.group(1)
            base_url = "https://ceplangobpe-my.sharepoint.com/personal/msuarez_ceplan_gob_pe/Documents/Aplicaciones/Microsoft Forms/Registro de Asistencia Técnica Participación de la/Question/"
            return base_url + archivo
    
    # Enlaces normales: eliminamos parámetros adicionales
    url_limpia = re.sub(r'[?&].*', '', url_decodificada)
    return url_limpia

# Normalizamos y separamos los enlaces múltiples
df_encuesta = normalize_and_split_links(df_encuesta, 'Adjuntar programa')
df_encuesta['Adjuntar programa'] = df_encuesta['Adjuntar programa'].apply(limpiar_url)

# Limpieza final
df_simplificado = df_encuesta.reset_index(drop=True)
df_simplificado.index += 1
df_simplificado.head()


,Id,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Seleccione la actividad operativa o tema relacionado,Especialista de la DNPE a cargo,Fecha de ejecución de la actividad,Fecha de fin de la actividad (si aplica),Modalidad,...,Número de espacios de acompañamiento y/o revisión durante el mes,Dirección de la persona de contacto/coordinación1,Nombre de la persona de contacto/coordinación1,N° de informe emitido mediante SGD1,Dificultades presentadas1,Oportunidades de mejora1,Información adicional de la asistencia técnica / capacitación y/o difusión y/o exposición1,Adjuntar programa,Adjuntar ppts,Adjuntar otras evidencias
1,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Enrique Del Águila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
2,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Enrique Del Águila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
3,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,3.0,DNCP,Isabel Rocio Uriona Diaz,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
4,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,3.0,DNCP,Isabel Rocio Uriona Diaz,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN
5,6,2024-06-25 00:02:35,2024-06-25 00:11:11,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-25,NaT,Virtual,...,1.0,DNCP,Roy Gerson Muñoz Gomez,NaN,Disponibilidad de tiempo limitado/insuficiente;,Se recomienda que la revisión del Proyecto de ...,Se dejo comentario al documento que contiene e...,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN


## 1.4. Funciones para limpiar los adjuntos (metadata)

In [11]:
sharepoint_metadata_df = pd.DataFrame(sharepoint_metadata_prueba)
df_decodificado = df_simplificado

# Decodificamos las URLs en 'Adjuntar programa' para hacerlas legibles
df_decodificado['Adjuntar programa'] = df_simplificado['Adjuntar programa'].apply(urllib.parse.unquote)

# Preparamos los enlaces completos del diccionario para compararlos
base_url = "https://ceplangobpe-my.sharepoint.com"
sharepoint_metadata_df['full_url'] = base_url + sharepoint_metadata_df['server_relative_url']
# Seleccionarmos columnas necesarias
sharepoint_metadata_prueba_reducido = sharepoint_metadata_df[['full_url', 'name', 'time_created']]

# Mostramos las URLs para comparar (el orden no es el mismo)
print("URL SharePoint (Codificada):", sharepoint_metadata_prueba_reducido.loc[4, "full_url"])
print("URL Encuesta (Decodificada):", df_decodificado.loc[4, "Adjuntar programa"])


URL SharePoint (Codificada): https://ceplangobpe-my.sharepoint.com/personal/msuarez_ceplan_gob_pe/Documents/Aplicaciones/Microsoft Forms/Registro de Asistencia Técnica Participación de la/Question/Reunión sobre convenio de Contraloría y Cepla_Milagros Estrada Ram.pdf
URL Encuesta (Decodificada): https://ceplangobpe-my.sharepoint.com/personal/msuarez_ceplan_gob_pe/Documents/Aplicaciones/Microsoft Forms/Registro de Asistencia Técnica Participación de la/Question/Correo sobre revisión de la propuesta de camb_Milagros Estrada Ram.pdf


## 1.5. Creación de la base de datos final

In [12]:
# Hacemos la unión por el enlace completo
df_merged = pd.merge(df_decodificado, sharepoint_metadata_prueba_reducido, how='left', left_on='Adjuntar programa', right_on='full_url')

df_merged.head(6)

,Id,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Seleccione la actividad operativa o tema relacionado,Especialista de la DNPE a cargo,Fecha de ejecución de la actividad,Fecha de fin de la actividad (si aplica),Modalidad,...,N° de informe emitido mediante SGD1,Dificultades presentadas1,Oportunidades de mejora1,Información adicional de la asistencia técnica / capacitación y/o difusión y/o exposición1,Adjuntar programa,Adjuntar ppts,Adjuntar otras evidencias,full_url,name,time_created
0,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Enrique Del Águila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,iv. OFICIO-000286-2024-PCD HUARAL_Alberto del ...,2024-06-24 20:52:02
1,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Enrique Del Águila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,Dia02_CEPLAN_HUARAL_ANALISIS PROSPECTIVO_Alber...,2024-06-24 20:52:15
2,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,Correo con las revisiones y coordinaciones de_...,2024-06-25 05:02:12
3,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,Correo de observaciones al proyecto PESEM MID_...,2024-06-25 05:02:12
4,6,2024-06-25 00:02:35,2024-06-25 00:11:11,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-25,NaT,Virtual,...,NaN,Disponibilidad de tiempo limitado/insuficiente;,Se recomienda que la revisión del Proyecto de ...,Se dejo comentario al documento que contiene e...,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,Anexo 1 - Estrategia Nacionl ante el Cambio C_...,2024-06-25 05:08:38
5,6,2024-06-25 00:02:35,2024-06-25 00:11:11,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-25,NaT,Virtual,...,NaN,Disponibilidad de tiempo limitado/insuficiente;,Se recomienda que la revisión del Proyecto de ...,Se dejo comentario al documento que contiene e...,https://ceplangobpe-my.sharepoint.com/personal...,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,Copia de Anexo 1_ENCC_REV_DNPE_ME250324_Milagr...,2024-06-25 05:08:33


## 1.6. Guardar como excel

In [3]:
excel_path = os.path.join(DOWNLOAD_PATH, "no_clasificados", "Registro de Participación con adjuntos_v3.xlsx")
#df_merged.to_excel(excel_path, index=False)

df_merged = pd.read_excel(excel_path)
df_merged.head()

,Id,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Seleccione la actividad operativa o tema relacionado,Especialista de la DNPE a cargo,Fecha de ejecución de la actividad,Fecha de fin de la actividad (si aplica),Modalidad,...,Dirección de la persona de contacto/coordinación1,Nombre de la persona de contacto/coordinación1,N° de informe emitido mediante SGD1,Dificultades presentadas1,Oportunidades de mejora1,Información adicional de la asistencia técnica / capacitación y/o difusión y/o exposición1,Adjuntar programa,full_url,name,time_created
0,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Alberto Del Aguila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,https://ceplangobpe-my.sharepoint.com/personal...,iv. OFICIO-000286-2024-PCD HUARAL_Alberto del ...,2024-06-24 20:52:02
1,4,2024-06-24 15:44:15,2024-06-24 15:52:35,adelaguila@ceplan.gob.pe,Alberto del Aguila Alfaro,Instrumentos técnicos en prospectiva,Alberto Del Aguila,2024-06-21,NaT,Presencial,...,NaN,NaN,NaN,NaN,NaN,NaN,https://ceplangobpe-my.sharepoint.com/personal...,https://ceplangobpe-my.sharepoint.com/personal...,Dia02_CEPLAN_HUARAL_ANALISIS PROSPECTIVO_Alber...,2024-06-24 20:52:15
2,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,DNCP,Isabel Rocio Uriona Diaz,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,https://ceplangobpe-my.sharepoint.com/personal...,Correo con las revisiones y coordinaciones de_...,2024-06-25 05:02:12
3,5,2024-06-24 23:27:22,2024-06-25 00:02:17,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-21,NaT,Virtual,...,DNCP,Isabel Rocio Uriona Diaz,NaN,Los comentarios señalados en las primeras revi...,Trabajar con documento en línea y respondiendo...,"El 29 de marzo de 2024, se dio VB al informe t...",https://ceplangobpe-my.sharepoint.com/personal...,https://ceplangobpe-my.sharepoint.com/personal...,Correo de observaciones al proyecto PESEM MID_...,2024-06-25 05:02:12
4,6,2024-06-25 00:02:35,2024-06-25 00:11:11,mestrada@ceplan.gob.pe,Milagros Estrada Ramos,Asistencia técnica (Políticas y planes),Milagros Estrada,2024-03-25,NaT,Virtual,...,DNCP,Roy Gerson Muñoz Gomez,NaN,Disponibilidad de tiempo limitado/insuficiente;,Se recomienda que la revisión del Proyecto de ...,Se dejo comentario al documento que contiene e...,https://ceplangobpe-my.sharepoint.com/personal...,https://ceplangobpe-my.sharepoint.com/personal...,Anexo 1 - Estrategia Nacionl ante el Cambio C_...,2024-06-25 05:08:38
